In [2]:
import sys,os
sys.path.append("..")
import django
django.setup()
import datetime
import pandas as pd
from django.conf import settings
from tw_data.stock_crawlers import *
from tw_data.models import *
from etl.import_sql import *
import logging
import datetime

logging.basicConfig(level=logging.INFO)

In [3]:
def replace_char(char,filter_list=None):
    if filter_list is None:
        filter_list=['-','（','）','/','－','(',')','─','∕']
    for c in filter_list:
        char=char.replace(c,'_').lower()
    char=char.replace(' ','').replace('__','_')
    if char[-1] == '_':
        char=char[:-1]
    return char



In [5]:
def tej_fundamental_clean(file_route='/Users/benbilly3/Downloads/fundamental (1).pkl'):
    df=pd.read_pickle(file_route)
    df=df.astype(str)
    df=df.reset_index()
    df = df.rename(columns={'證券名稱': 'stock_id','日期':'date'})
#     exist_date=list(TejFundamental.objects.values_list('date',flat=True).distinct())
#     df=df[~df['date'].isin(exist_date)]
    if len(df)<0:
        return None
    df['stock_name']=df['stock_id'].apply(lambda s:s[s.index(' ')+1:])
    df['stock_id']=df['stock_id'].apply(lambda s:s[:s.index(' ')])
    new_columns=[replace_char(i) for i in df.columns]
    # for i in new_columns:
    #     print(i)
    df.columns=new_columns
    float_columns=new_columns[new_columns.index('現金及約當現金'):new_columns.index('現金股利率')+1]
    float_columns.extend(new_columns[new_columns.index('透過其他綜合損益按公允價值衡量之金融資產_流動'):new_columns.index('stock_name')])

    for i in float_columns:
        df[i]=pd.to_numeric(df[i],errors='coerce')
    return df

df=tej_fundamental_clean()
df

,stock_id,date,上市別,季別,合併_y_n,單季_q_單半年_h,月份,幣別,現金及約當現金,透過損益按公允價值衡量之金融資產_流動,...,期中特別股_現金股利,期中特別股_股票股利,期末特別股_現金股利,期末特別股_股票股利,淨負債,營業利益_實收資本比,稅前純益_實收資本,季底應收帳款_營收tse,季底存貨_營收tse,stock_name
0,1101,2005-08-31,TSE,2,Y,H,6,NTD,10711519.0,3956662.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,台泥
1,1491,2005-08-31,,2,N,H,6,NTD,3248.0,50.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,東榮工
2,1566,2005-08-31,,2,N,H,6,NTD,13253.0,2898.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,捷邦
3,1787,2005-08-31,,2,Y,H,6,NTD,73774.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,福盈科
4,1807,2005-08-31,,2,Y,H,6,NTD,22534.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,羅馬
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96407,8131,2020-11-14,TSE,3,N,Q,9,NTD,1400065.0,0.0,...,NaN,NaN,NaN,NaN,-3117308.0,8.23,9.51,68.40,24.00,福懋科
96408,8150,2020-11-14,TSE,3,Y,Q,9,NTD,4788223.0,217820.0,...,NaN,NaN,NaN,NaN,4028025.0,9.87,7.41,81.56,35.81,南茂
96409,8163,2020-11-14,TSE,3,Y,Q,9,NTD,2226214.0,339826.0,...,NaN,NaN,NaN,NaN,1620018.0,12.09,13.83,112.29,62.35,達方
96410,M9900,2020-11-14,-,3,Y,Q,9,NTD,200825834.0,18310959.0,...,NaN,NaN,NaN,NaN,NaN,12.91,17.78,NaN,NaN,其他類指數


In [6]:
df.to_pickle('/Users/benbilly3/finlab/sql_interface/pickle_table/tw_data/tej_fundamental.pkl')

# Generate SQL code

In [ ]:
test=df.iloc[0]
for i in range(len(test)):
    v=test.iloc[i]
    print(v,str(type(v)))

In [ ]:
def sql_code_generate(col,v):
    format_=''
    if str(type(v))=="<class 'pandas._libs.tslibs.timestamps.Timestamp'>":
        format_=f'  `{col}` DATETIME NOT NULL,'
    elif str(type(v))=="<class 'float'>":
        format_=f'  `{col}` FLOAT NULL,'
    elif str(type(v))=="<class 'numpy.float64'>":
        format_=f'  `{col}` FLOAT NULL,'
    elif str(type(v))=="<class 'numpy.int64'>":
        format_=f'  `{col}`  INT NULL,'
    elif str(type(v))=="<class 'str'>":   
        format_=f'  `{col}` VARCHAR(16) NULL,'   
    return format_


In [ ]:
for i in range(len(test)):
    col=list(test.index)
    v=test.iloc[i]
    col_=col[i]
    print(sql_code_generate(col_,v))

# SQLCODE

# 批次匯入

In [6]:
batchly_import_df(df, TejFundamental, range_=10000)

INFO:etl.import_sql:Finish!<class 'tw_data.models.TejFundamental'> date:from 2005-08-31 to 2008-04-30 bulk_create:10000
INFO:etl.import_sql:Finish!<class 'tw_data.models.TejFundamental'> date:from 2005-08-31 to 2008-04-30 bulk_update:0
INFO:etl.import_sql:Finish!<class 'tw_data.models.TejFundamental'> date:from 2008-04-30 to 2010-03-31 bulk_create:10000
INFO:etl.import_sql:Finish!<class 'tw_data.models.TejFundamental'> date:from 2008-04-30 to 2010-03-31 bulk_update:0
INFO:etl.import_sql:Finish!<class 'tw_data.models.TejFundamental'> date:from 2010-03-31 to 2011-08-31 bulk_create:10000
INFO:etl.import_sql:Finish!<class 'tw_data.models.TejFundamental'> date:from 2010-03-31 to 2011-08-31 bulk_update:0
INFO:etl.import_sql:Finish!<class 'tw_data.models.TejFundamental'> date:from 2011-08-31 to 2013-03-31 bulk_create:10000
INFO:etl.import_sql:Finish!<class 'tw_data.models.TejFundamental'> date:from 2011-08-31 to 2013-03-31 bulk_update:0
INFO:etl.import_sql:Finish!<class 'tw_data.models.TejFun